In [1]:
storage_path='../'

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import os
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator

In [3]:
input_file=os.path.join(storage_path,'data','influencer_sample010525_weekly.csv')

output_file_summary=os.path.join(storage_path,'output',f"summary_df_{dt.datetime.now():%y%m%d%H%M}.csv")

output_file_complete=os.path.join(storage_path,'output',f"summary_complete_df_{dt.datetime.now():%y%m%d%H%M}.csv")

In [4]:

h=4
window=24
test=False

sources=['instagram-content','twitter-tweets','tiktok-content','facebook-content','youtube-content',
 'tiktok-content-oauth','instagram-content-oauth','tiktok-content-public']
#models_eval=['AutoARIMA', 'Theta', 'ETS', 'Prophet', 'AutoREG(2)', 'AutoREG(3)', 'AutoREG(4)', 'AutoREG(5)', 'AR(2)', 'AR(3)', 'AR(4)', 'AR(5)', 'MA(2)', 'MA(3)', 'MA(4)', 'MA(5)', 'Neural Forecast LTSM', 'Neural Forecast RNN']
models_eval=[ 'Theta']
df= pd.read_csv(input_file).query('source in @sources').reset_index(drop=True)
df['published_date'] = pd.to_datetime(df['published_date_w'])
df


,published_date_w,channel_uid,n_posts,engagements_sum,video_views_sum,shares_sum,comments_sum,first_audience,last_audience,source,tier,mean_audience,engagements_rate_sd,posts_per_day,normalized_engagement_rate,min_published_date_w,audience_growth,first_non_zero_growth_date,published_date
0,2017-09-10,d0334d17ea133c5aac89e40ce50404d8,5,90,0,0,2,29295.0,29297.0,instagram-content,"10,000–49,999",2.929580e+04,0.000655,0.714286,0.003072,2016-01-24,0.000068,2017-09-10,2017-09-10
1,2017-09-17,d0334d17ea133c5aac89e40ce50404d8,9,246,1153,0,15,29297.0,29289.0,instagram-content,"10,000–49,999",2.929256e+04,0.001195,1.285714,0.008398,2016-01-24,-0.000273,2017-09-10,2017-09-17
2,2017-10-01,d0334d17ea133c5aac89e40ce50404d8,4,166,0,0,5,29280.0,29280.0,instagram-content,"10,000–49,999",2.928000e+04,0.001152,0.571429,0.005669,2016-01-24,-0.000307,2017-09-10,2017-10-01
3,2017-10-08,d0334d17ea133c5aac89e40ce50404d8,5,98,0,0,4,29280.0,29270.0,instagram-content,"10,000–49,999",2.927600e+04,0.000583,0.714286,0.003347,2016-01-24,-0.000342,2017-09-10,2017-10-08
4,2017-10-22,d0334d17ea133c5aac89e40ce50404d8,2,15,0,0,2,29298.0,29298.0,instagram-content,"10,000–49,999",2.929800e+04,0.000024,0.285714,0.000512,2016-01-24,0.000957,2017-09-10,2017-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19609,2025-03-16,1c8afac3e44638cab68212f6f634f6d8,1,24699,0,0,296,2473766.0,2473766.0,instagram-content,"500,000+",2.473766e+06,NaN,0.142857,0.009984,2019-08-04,0.000000,2021-10-31,2025-03-16
19610,2025-03-16,6833f3e4b5753906a72ff0fbbdb53151,2,223,0,6,19,317000.0,317000.0,tiktok-content,"100,000–499,999",3.170000e+05,0.000199,0.285714,0.000703,2019-11-24,0.000000,2020-10-25,2025-03-16
19611,2025-03-16,a8b4713fd3393bbcaa4df879a42d1efc,1,19,0,0,0,9782.0,9782.0,instagram-content,"1,000–9,999",9.782000e+03,NaN,0.142857,0.001942,2017-12-24,0.000000,2020-04-05,2025-03-16
19612,2025-03-16,e04c6702e00a321a8f98624ffe7fd23e,2,781,0,154,15,1900000.0,1900000.0,tiktok-content,"500,000+",1.900000e+06,0.000185,0.285714,0.000411,2021-10-31,0.000000,2022-03-20,2025-03-16


In [5]:
df['source'].value_counts()

source
instagram-content    14224
tiktok-content        3357
facebook-content      1082
youtube-content        951
Name: count, dtype: int64

In [6]:
ids_test=['b95a899244573cf38802aa520ef3a82b']
if test:
    df_temp=df.query('channel_uid in @ids_test')
else:
    df_temp=df.copy()
df_temp['source'].unique()

array(['instagram-content', 'facebook-content', 'youtube-content',
       'tiktok-content'], dtype=object)

In [7]:
df_temp

,published_date_w,channel_uid,n_posts,engagements_sum,video_views_sum,shares_sum,comments_sum,first_audience,last_audience,source,tier,mean_audience,engagements_rate_sd,posts_per_day,normalized_engagement_rate,min_published_date_w,audience_growth,first_non_zero_growth_date,published_date
0,2017-09-10,d0334d17ea133c5aac89e40ce50404d8,5,90,0,0,2,29295.0,29297.0,instagram-content,"10,000–49,999",2.929580e+04,0.000655,0.714286,0.003072,2016-01-24,0.000068,2017-09-10,2017-09-10
1,2017-09-17,d0334d17ea133c5aac89e40ce50404d8,9,246,1153,0,15,29297.0,29289.0,instagram-content,"10,000–49,999",2.929256e+04,0.001195,1.285714,0.008398,2016-01-24,-0.000273,2017-09-10,2017-09-17
2,2017-10-01,d0334d17ea133c5aac89e40ce50404d8,4,166,0,0,5,29280.0,29280.0,instagram-content,"10,000–49,999",2.928000e+04,0.001152,0.571429,0.005669,2016-01-24,-0.000307,2017-09-10,2017-10-01
3,2017-10-08,d0334d17ea133c5aac89e40ce50404d8,5,98,0,0,4,29280.0,29270.0,instagram-content,"10,000–49,999",2.927600e+04,0.000583,0.714286,0.003347,2016-01-24,-0.000342,2017-09-10,2017-10-08
4,2017-10-22,d0334d17ea133c5aac89e40ce50404d8,2,15,0,0,2,29298.0,29298.0,instagram-content,"10,000–49,999",2.929800e+04,0.000024,0.285714,0.000512,2016-01-24,0.000957,2017-09-10,2017-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19609,2025-03-16,1c8afac3e44638cab68212f6f634f6d8,1,24699,0,0,296,2473766.0,2473766.0,instagram-content,"500,000+",2.473766e+06,NaN,0.142857,0.009984,2019-08-04,0.000000,2021-10-31,2025-03-16
19610,2025-03-16,6833f3e4b5753906a72ff0fbbdb53151,2,223,0,6,19,317000.0,317000.0,tiktok-content,"100,000–499,999",3.170000e+05,0.000199,0.285714,0.000703,2019-11-24,0.000000,2020-10-25,2025-03-16
19611,2025-03-16,a8b4713fd3393bbcaa4df879a42d1efc,1,19,0,0,0,9782.0,9782.0,instagram-content,"1,000–9,999",9.782000e+03,NaN,0.142857,0.001942,2017-12-24,0.000000,2020-04-05,2025-03-16
19612,2025-03-16,e04c6702e00a321a8f98624ffe7fd23e,2,781,0,154,15,1900000.0,1900000.0,tiktok-content,"500,000+",1.900000e+06,0.000185,0.285714,0.000411,2021-10-31,0.000000,2022-03-20,2025-03-16


In [8]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid')

In [9]:
multi_eval.run(h=h, initial_window=window, step_length=1,models=models_eval,metric_cols=['mean_audience'])

summary_df = multi_eval.get_summary()
summary_complete_df = multi_eval.get_summary_complete()

Evaluating models:   0%|          | 0/178 [00:00<?, ?it/s]

2025-05-18 14:11:00,673 Evaluating channel: d0334d17ea133c5aac89e40ce50404d8
2025-05-18 14:11:00,682 Evaluating metric: mean_audience
2025-05-18 14:11:00,683 Evaluating Theta for d0334d17ea133c5aac89e40ce50404d8/mean_audience


Evaluating models:   1%|          | 1/178 [00:12<37:55, 12.86s/it]

2025-05-18 14:11:13,531 1 of 178
2025-05-18 14:11:13,532 Evaluating channel: a41e989adad13e3dbb0fc6260bf29c42
2025-05-18 14:11:13,542 Evaluating metric: mean_audience
2025-05-18 14:11:13,543 Evaluating Theta for a41e989adad13e3dbb0fc6260bf29c42/mean_audience


Evaluating models:   1%|          | 2/178 [00:18<25:00,  8.52s/it]

...

Evaluating models:  76%|███████▋  | 136/178 [06:54<00:23,  1.80it/s]

2025-05-18 14:17:55,214 136 of 178
2025-05-18 14:17:55,215 Evaluating channel: 92a43c6283de32d8af1196b92b8b6658
2025-05-18 14:17:55,224 Evaluating metric: mean_audience
2025-05-18 14:17:55,226 Evaluating Theta for 92a43c6283de32d8af1196b92b8b6658/mean_audience


/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/esteban/r

2025-05-18 14:17:55,921 137 of 178
2025-05-18 14:17:55,922 Evaluating channel: a9b4613c4b713eefb9dc7e4b8646dc25
2025-05-18 14:17:55,931 Evaluating metric: mean_audience
2025-05-18 14:17:55,933 Skipping a9b4613c4b713eefb9dc7e4b8646dc25 / mean_audience (not enough data)
2025-05-18 14:17:55,934 Evaluating channel: 8d620284a52c30d69e286a4eee24a8ec
2025-05-18 14:17:55,942 Evaluating metric: mean_audience
2025-05-18 14:17:55,943 Skipping 8d620284a52c30d69e286a4eee24a8ec / mean_audience (not enough data)
2025-05-18 14:17:55,945 Evaluating channel: 10c4b328c0d03ef0b36a45a708ee2214
2025-05-18 14:17:55,954 Evaluating metric: mean_audience
2025-05-18 14:17:55,957 Skipping 10c4b328c0d03ef0b36a45a708ee2214 / mean_audience (not enough data)
2025-05-18 14:17:55,958 Evaluating channel: 49ca0a9bafd833b795d88898aa54d7e1
2025-05-18 14:17:55,967 Evaluating metric: mean_audience
2025-05-18 14:17:55,969 Skipping 49ca0a9bafd833b795d88898aa54d7e1 / mean_audience (not enough data)
2025-05-18 14:17:55,971 Evalu

Evaluating models:  77%|███████▋  | 137/178 [06:55<02:04,  3.03s/it]


In [ ]:

summary_df.to_csv(output_file_summary, index=False)
summary_df

,model_name,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,test_MeanSquaredError,channel_uid,metric_name,source
0,Theta,52.098971,0.001903,58.235691,d0334d17ea133c5aac89e40ce50404d8,mean_audience,instagram-content
1,Theta,21.815450,0.000759,25.119896,a41e989adad13e3dbb0fc6260bf29c42,mean_audience,facebook-content
2,Theta,3883.573979,0.026119,4512.417816,df6284767fef3c05b2bcb2e9c9cf5277,mean_audience,instagram-content
3,Theta,44.269394,0.021709,47.090244,f477a13910a236b7b4bb1381ea6630bc,mean_audience,youtube-content
4,Theta,1516.892791,0.013473,1667.339544,f7674e4b4d7d3e3c90c7490028b5345a,mean_audience,instagram-content
...,...,...,...,...,...,...,...
132,Theta,23.822022,0.001759,26.381006,b01d15b00144339782f8e9fef89dceda,mean_audience,instagram-content
133,Theta,1703.815984,0.011288,1973.981013,f84b3aaef6583cb9bd11ec8a859e5810,mean_audience,youtube-content
134,Theta,23.057228,0.003124,25.875698,cd533e37cf5e3e6290ce68a41be6fd53,mean_audience,instagram-content
135,Theta,15.289687,0.001362,17.195505,6d58b2d72bed32af821510c6aef723ac,mean_audience,instagram-content


In [11]:

summary_complete_df.to_csv(output_file_complete, index=False)
summary_complete_df

,test_MeanAbsoluteError,test_MeanSquaredError,test_MeanAbsolutePercentageError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred,model_name,channel,metric_name,source
0,19.299015,25.050694,0.000656,1.402039,0.041452,24,23,0 29295.800000 1 29292.555556 2 29...,24 29403.571429 25 29403.333333 26 29...,24 29394.015529 25 29396.123293 26 29...,Theta,d0334d17ea133c5aac89e40ce50404d8,mean_audience,instagram-content
1,18.135846,23.547366,0.000616,1.244259,0.026191,25,24,0 29295.800000 1 29292.555556 2 29...,25 29403.333333 26 29412.000000 27 29...,25 29408.167513 26 29410.465556 27 29...,Theta,d0334d17ea133c5aac89e40ce50404d8,mean_audience,instagram-content
2,26.255487,29.344034,0.000892,0.619733,0.021244,26,25,0 29295.800000 1 29292.555556 2 29...,26 29412.000000 27 29447.000000 28 29...,26 29408.187341 27 29410.614345 28 29...,Theta,d0334d17ea133c5aac89e40ce50404d8,mean_audience,instagram-content
3,28.677641,28.872472,0.000974,0.691436,0.021138,27,26,0 29295.800000 1 29292.555556 2 29...,27 29447.000000 28 29447.000000 29 29...,27 29417.088967 28 29419.633451 29 29...,Theta,d0334d17ea133c5aac89e40ce50404d8,mean_audience,instagram-content
4,6.584299,7.934200,0.000224,0.717517,0.063959,28,27,0 29295.800000 1 29292.555556 2 29...,28 29447.000000 29 29446.333333 30 29...,28 29452.498068 29 29455.247102 30 29...,Theta,d0334d17ea133c5aac89e40ce50404d8,mean_audience,instagram-content
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15362,1427.858875,1487.375261,0.001298,0.132483,0.041930,38,37,0 1011429.75 1 1060381.50 2 106556...,38 1100000.0 39 1100000.0 40 1100000....,38 1.100869e+06 39 1.101242e+06 40 1....,Theta,92a43c6283de32d8af1196b92b8b6658,mean_audience,tiktok-content
15363,1359.868368,1416.550757,0.001236,0.092306,0.023611,39,38,0 1011429.75 1 1060381.50 2 106556...,39 1100000.0 40 1100000.0 41 1100000....,39 1.100828e+06 40 1.101182e+06 41 1....,Theta,92a43c6283de32d8af1196b92b8b6658,mean_audience,tiktok-content
15364,1296.603819,1350.649198,0.001179,0.052495,0.020337,40,39,0 1011429.75 1 1060381.50 2 106556...,40 1100000.0 41 1100000.0 42 1100000....,40 1.100789e+06 41 1.101127e+06 42 1....,Theta,92a43c6283de32d8af1196b92b8b6658,mean_audience,tiktok-content
15365,1237.638844,1289.226430,0.001125,0.102989,0.038734,41,40,0 1011429.75 1 1060381.50 2 106556...,41 1100000.0 42 1100000.0 43 1100000....,41 1.100753e+06 42 1.101076e+06 43 1....,Theta,92a43c6283de32d8af1196b92b8b6658,mean_audience,tiktok-content
